## Feature Extraction HSV

In [1]:
import cv2
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, entropy

In [2]:
# Fungsi untuk menghitung mean, skewness, entropy, dan standard deviation dari komponen HSV
def calculate_hsv_features(image):
    # Konversi citra ke ruang warna HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Pisahkan komponen H, S, dan V
    h, s, v = cv2.split(hsv_image)
    
    # Hitung mean (rata-rata) dari setiap komponen
    h_mean = np.mean(h)
    s_mean = np.mean(s)
    v_mean = np.mean(v)
    
    # Hitung skewness (kemiringan) dari setiap komponen
    h_skewness = skew(h.flatten())
    s_skewness = skew(s.flatten())
    v_skewness = skew(v.flatten())
    
    # Hitung entropy (entropi) dari setiap komponen
    h_entropy = entropy(h.flatten())
    s_entropy = entropy(s.flatten())
    v_entropy = entropy(v.flatten())
    
    # Hitung standard deviation (deviasi standar) dari setiap komponen
    h_std = np.std(h)
    s_std = np.std(s)
    v_std = np.std(v)
    
    return {
        "H_mean": h_mean,
        "S_mean": s_mean,
        "V_mean": v_mean,
        "H_skewness": h_skewness,
        "S_skewness": s_skewness,
        "V_skewness": v_skewness,
        "H_entropy": h_entropy,
        "S_entropy": s_entropy,
        "V_entropy": v_entropy,
        "H_std": h_std,
        "S_std": s_std,
        "V_std": v_std
    }


# Fungsi untuk meresize citra
def resize_image(input_path, output_path, new_width, new_height):
    # Baca citra
    image = cv2.imread(input_path)

    # Resize citra ke ukuran yang diinginkan
    resized_image = cv2.resize(image, (new_width, new_height))

    # Simpan citra yang telah diresize
    cv2.imwrite(output_path, resized_image)

# Fungsi utama untuk menghitung fitur HSV dari semua gambar dalam folder dengan resize
def calculate_features_in_folder_and_resize(folder_path, output_folder, new_width, new_height):
    all_features = []

    # Dapatkan daftar semua file citra dalam folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg'))]

    for image_file in image_files:
        input_path = os.path.join(folder_path, image_file)
        image = cv2.imread(input_path)

        if image is not None:
            # Hitung fitur HSV
            features = calculate_hsv_features(image)

            # Simpan hasil perhitungan fitur HSV
            all_features.append(features)

            # Meresize gambar
            output_path = os.path.join(output_folder, image_file)
            resize_image(input_path, output_path, new_width, new_height)

    return all_features

# Fungsi utama untuk menghitung fitur HSV dari semua kelas folder dan resize gambar
def calculate_features_in_all_classes_and_resize(main_folder, output_csv_path, new_width, new_height):
    all_features = []
    all_labels = []

    # Dapatkan daftar semua folder kelas dalam direktori utama
    class_folders = [os.path.join(main_folder, folder) for folder in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, folder))]

    for class_folder in class_folders:
        class_name = os.path.basename(class_folder)

        # Buat folder output untuk hasil resize
        output_class_folder = os.path.join(main_folder, class_name + "_resized")
        os.makedirs(output_class_folder, exist_ok=True)

        # Hitung fitur HSV dan resize gambar
        class_features = calculate_features_in_folder_and_resize(class_folder, output_class_folder, new_width, new_height)
        all_features.extend(class_features)
        all_labels.extend([class_name] * len(class_features))

    # Konversi hasil perhitungan ke DataFrame
    df = pd.DataFrame(all_features)
    df["label"] = all_labels

    # Simpan hasil ke dalam file CSV
    df.to_csv(output_csv_path, index=False)

# Save File Dataset yang udah di Resize
main_folder = '../Dataset_resize/'  
output_csv_path = '../notebook/result/output_feature_extraction_HSV.csv'  
new_width = 300  # Lebar citra
new_height = 300  # Tinggi citra

calculate_features_in_all_classes_and_resize(main_folder, output_csv_path, new_width, new_height)


In [3]:
file_path = '../notebook/result/output_feature_extraction_HSV.csv'
df = pd.read_csv(file_path)
df.head()

,H_mean,S_mean,V_mean,H_skewness,S_skewness,V_skewness,H_entropy,S_entropy,V_entropy,H_std,S_std,V_std,label
0,112.918556,25.083630,233.556561,-8.984022,5.166388,-5.116159,15.930703,15.777923,15.926208,8.421821,18.594424,27.223275,Skala0
1,102.019578,6.076716,239.601118,-2.936337,8.822647,-5.048020,13.755722,13.160871,13.811461,26.847181,12.564507,19.617937,Skala0
2,107.860592,49.071800,219.021018,-4.309712,5.606899,-4.171041,13.813757,13.791267,13.794408,6.033221,12.621105,37.301510,Skala0
3,107.707690,42.995309,214.469718,-6.700319,4.903321,-3.790741,13.812133,13.777719,13.802779,7.833227,13.849444,30.174435,Skala0
4,111.507190,21.193654,218.849183,-5.202897,5.463856,-4.667438,13.812049,13.625900,13.804734,8.537049,18.484920,27.969296,Skala0
